In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import os
import kagglehub

In [2]:
# 1. Загрузка датасета
path = kagglehub.dataset_download("imbikramsaha/food11")
data_dir = os.path.join(path, "food11")

100%|██████████| 519M/519M [00:24<00:00, 22.4MB/s]

Extracting files...


In [3]:
# 2. Настройка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# 3. Размеры входа для каждой модели
input_sizes = {"b0": 224, "b1": 240, "b2": 260}

In [5]:
# 4. Функция подготовки данных
def get_dataloaders(input_size):
    transform = transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    dataset = ImageFolder(root=data_dir, transform=transform)
    num_classes = len(dataset.classes)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    return train_loader, val_loader, num_classes

In [6]:
# 5. Функция создания модели
def get_efficientnet(version: str, num_classes: int):
    if version == "b0":
        model = models.efficientnet_b0(pretrained=True)
    elif version == "b1":
        model = models.efficientnet_b1(pretrained=True)
    elif version == "b2":
        model = models.efficientnet_b2(pretrained=True)
    else:
        raise ValueError("Unknown EfficientNet version")
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model.to(device)

In [7]:
# 6. Функции обучения и оценки
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    loop = tqdm(loader, desc="Training", leave=False)
    for inputs, targets in loop:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    return running_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == targets).sum().item()
            total += targets.size(0)
    return 100 * correct / total

In [8]:
# 7. Цикл обучения и сравнения
criterion = nn.CrossEntropyLoss()
results = {}

for version in ["b0", "b1", "b2"]:
    print(f"\n Обучение EfficientNet-{version.upper()}")
    input_size = input_sizes[version]
    train_loader, val_loader, num_classes = get_dataloaders(input_size)
    model = get_efficientnet(version, num_classes)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(5):  # можно увеличить до 10
        print(f"Epoch {epoch+1}/5")
        train_loss = train_epoch(model, train_loader, criterion, optimizer)
        val_acc = evaluate(model, val_loader)
        print(f"Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")

    results[version] = val_acc

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



 Обучение EfficientNet-B0
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 202MB/s]


Epoch 1/5


Loss: 0.3536 | Val Acc: 89.95%
Epoch 2/5


...

Loss: 0.3444 | Val Acc: 90.64%
Epoch 2/5


Loss: 0.2328 | Val Acc: 89.09%
Epoch 3/5


Loss: 0.1257 | Val Acc: 88.82%
Epoch 4/5


Loss: 0.0575 | Val Acc: 87.64%
Epoch 5/5


Loss: 0.0402 | Val Acc: 87.73%


In [9]:
# 8. Вывод результатов
print("\n Сравнение моделей:")
for version, acc in results.items():
    print(f"EfficientNet-{version.upper()}: {acc:.2f}% точность на валидации")


 Сравнение моделей:
EfficientNet-B0: 86.73% точность на валидации
EfficientNet-B1: 86.27% точность на валидации
EfficientNet-B2: 87.73% точность на валидации


Как видим, модели хорошо справляются на тренировочных данных, но хуже справляются с тестовыми, что говорит о переобучении. Модель, которая обучалась на гораздо большем датасете и в приницпе очень мощная легко переобучается на небольшом датасете вроде нашего.